In [17]:
# Import libraries.
import numpy as np
import pandas as pd
import requests
import json

In [18]:
# Define FourSquare credentials.
CLIENT_ID = 'YL4F00KRM2ATGYXNWMXR1FGN45UF0LTMGCSSNXSKQYXHJN5E' # your Foursquare ID
CLIENT_SECRET = 'OZB2DVDSN3SLHKJUOND3EIQL1UYSHZT1P0P0PJUHI2FZNWKD' # your Foursquare Secret
VERSION = '20180605'# Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YL4F00KRM2ATGYXNWMXR1FGN45UF0LTMGCSSNXSKQYXHJN5E
CLIENT_SECRET:OZB2DVDSN3SLHKJUOND3EIQL1UYSHZT1P0P0PJUHI2FZNWKD


### Chicago Food Venues
The code, below, finds food venues around Chicago.

In [19]:
# Grab as many food venues as possible around Chicago, IL.
# We need to query more than one starting long/lat point because limit per query is 100.
# Then we remove duplicates by venue_id.

# Initialize a dataframe to store Chicago food venues.
chicago_food_venues = pd.DataFrame(columns=['id','name','latitude','longitude','primary_category'])

# Define a function to get food venues for different longitude and latitude locations corresponding to different
# Chicago neighborhoods.
# The radius parameter is not provided so FourSquare gives a suggested radius based on density of venues.
# Get the food venues for any day and any time so it is not just limited to the day and time of query.
# Get the food venues that are not fast food by limiting to a price range of 2,3,4.
def get_food_venues(client_id, client_secret, version, ll):
    df = pd.DataFrame(columns=['id','name','latitude','longitude','primary_category'])
    explore_url = r'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={}&section={}&limit={}&time={}&day={}&price={}'.format(client_id, client_secret, version, ll, "food", "100","any","any","3,4")
    results = requests.get(explore_url).json()
    for place in results['response']['groups'][0]['items']:
        i = place['venue']['id']
        name = place['venue']['name']
        lat = place['venue']['location']['lat']
        lng = place['venue']['location']['lng']
        cat = place['venue']['categories'][0]['name']
        df = df.append({'id':i,'name':name,'latitude':lat,'longitude':lng,'primary_category':cat}, ignore_index=True)
    return df

In [4]:
# Loop over Chicago neighborhoods to get food venues near those neighborhoods.
neighborhoods = [
    '41.8786,-87.6251',# The Loop
    '41.8854,-87.6627',# West Loop
    '41.8674,-87.6275',# South Loop
    '41.8507,-87.6340',# Chinatown
    '41.9398,-87.6589',# Lake View
    '41.8924,-87.6341',# River North
    '41.9227,-87.6803',# Bucktown
    '41.8856,-87.6509',# Fulton Market
    '41.9255,-87.6488',# Lincoln Park
    '41.9665,-87.6533',# Uptown
    '41.9440,-87.6474',# Boystown
    '41.9231,-87.7093',# Logan Square
    '41.9491,-87.6565',# Wrigleyville
    '41.8927,-87.6200',# Streeterville
    '41.8993,-87.6639',# Noble Square
    '41.8994,-87.6845',# Ukranian Village
    '41.9067,-87.6253',# Gold Coast
    '41.9111,-87.6410',# Old Town
    '41.9105,-87.6776',# Wicker Park
    '41.8991,-87.7213',# Humboldt Park
    '41.8781,-87.6298'# All of Chicago
]
for neighborhood in neighborhoods:
    df = get_food_venues(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood)
    chicago_food_venues = chicago_food_venues.append(df, ignore_index=True)
chicago_food_venues

,id,name,latitude,longitude,primary_category
0,49e749fbf964a52086641fe3,The Gage,41.881202,-87.624481,Gastropub
1,55669b9b498ee34e5249ea61,Cindy's,41.881695,-87.624600,New American Restaurant
2,556b5527498e3bb54be5ff78,Cherry Circle Room,41.881994,-87.625050,American Restaurant
3,53d6f406498e54a8e8dfb7c4,Acanto,41.881495,-87.624500,Italian Restaurant
4,4ac7958cf964a52036b820e3,Terzo Piano,41.880443,-87.622431,Italian Restaurant
...,...,...,...,...,...
228,4c6b0b116d390f4708ed099a,The Florentine,41.879170,-87.633032,Italian Restaurant
229,49e749fbf964a52086641fe3,The Gage,41.881202,-87.624481,Gastropub
230,55669b9b498ee34e5249ea61,Cindy's,41.881695,-87.624600,New American Restaurant
231,556b5527498e3bb54be5ff78,Cherry Circle Room,41.881994,-87.625050,American Restaurant


In [7]:
# Drop duplicates by venue id.
chicago_food_venues.drop_duplicates(subset=['id'], inplace=True)
chicago_food_venues.reset_index(inplace=True)
chicago_food_venues.shape

(196, 7)

In [8]:
# Check the breakdown of category to make sure there is a good variety.
chicago_food_venues.groupby('primary_category').count()

,level_0,index,id,name,latitude,longitude
primary_category,,,,,,
American Restaurant,10,10,10,10,10,10
Argentinian Restaurant,1,1,1,1,1,1
BBQ Joint,1,1,1,1,1,1
Brazilian Restaurant,1,1,1,1,1,1
Burger Joint,1,1,1,1,1,1
Café,2,2,2,2,2,2
Chinese Restaurant,3,3,3,3,3,3
Deli / Bodega,1,1,1,1,1,1
Dumpling Restaurant,1,1,1,1,1,1


In [9]:
# Get most popoular Chicago, IL food venues.
most_popular = pd.DataFrame(columns={'id', 'name', 'latitude', 'longitude', 'primary_category', 'popularity_rating'})
details_url = "https://api.foursquare.com/v2/venues/explore/?near={}&sortByPopularity={}&price={}&section={}&limit={}&client_id={}&client_secret={}&v={}".format("Chicago, IL", "1", "3,4", "food", "25", CLIENT_ID, CLIENT_SECRET, VERSION)
results = requests.get(details_url).json()
for index, place in enumerate(results['response']['groups'][0]['items']):
    i = place['venue']['id']
    name = place['venue']['name']
    lat = place['venue']['location']['lat']
    lng = place['venue']['location']['lng']
    cat = place['venue']['categories'][0]['name']
    popularity_rating = index+1
    most_popular = most_popular.append({'id':i, 'name':name, 'latitude':lat, 'longitude':lng, 'primary_category':cat, 'popularity_rating':popularity_rating}, ignore_index=True)
most_popular

,name,popularity_rating,latitude,id,primary_category,longitude
0,Beatrix,1,41.894425,56290b27498e2c6aacd233fe,New American Restaurant,-87.622538
1,The Gage,2,41.881202,49e749fbf964a52086641fe3,Gastropub,-87.624481
2,The Dearborn,3,41.884415,5755b556498ea375c5d4baac,Gastropub,-87.629554
3,Shaw's Crab House,4,41.889959,4ad91beff964a5202e1821e3,Seafood Restaurant,-87.627065
4,Gibson Italia,5,41.886606,59deb72f061b514b32a062af,Italian Restaurant,-87.639593
5,"Joe's Seafood, Prime Steak & Stone Crab",6,41.891828,4a2ae23af964a52064961fe3,Seafood Restaurant,-87.625444
6,Siena Tavern,7,41.888881,50f58653e4b0e1058164dc7e,Italian Restaurant,-87.630272
7,Maple & Ash,8,41.901931,561af710498ef927dbf9b658,Steakhouse,-87.628476
8,Mon Ami Gabi,9,41.923927,49ebe487f964a52043671fe3,French Restaurant,-87.636615
9,Gibsons Bar & Steakhouse,10,41.901527,4ac7e2f3f964a52056ba20e3,Steakhouse,-87.627983


In [10]:
# Merge most popular food venues on to all food venues
chicago_food_venues = chicago_food_venues.merge(most_popular, how='outer', on=['id', 'name', 'latitude', 'longitude','primary_category'])
chicago_food_venues = chicago_food_venues.drop(columns=[['index','level_0']])

,level_0,id,name,latitude,longitude,primary_category,popularity_rating
0,0.0,49e749fbf964a52086641fe3,The Gage,41.881202,-87.624481,Gastropub,2
1,1.0,55669b9b498ee34e5249ea61,Cindy's,41.881695,-87.624600,New American Restaurant,23
2,2.0,556b5527498e3bb54be5ff78,Cherry Circle Room,41.881994,-87.625050,American Restaurant,NaN
3,3.0,53d6f406498e54a8e8dfb7c4,Acanto,41.881495,-87.624500,Italian Restaurant,NaN
4,4.0,4ac7958cf964a52036b820e3,Terzo Piano,41.880443,-87.622431,Italian Restaurant,NaN
...,...,...,...,...,...,...,...
195,195.0,4c6b0b116d390f4708ed099a,The Florentine,41.879170,-87.633032,Italian Restaurant,NaN
196,NaN,5755b556498ea375c5d4baac,The Dearborn,41.884415,-87.629554,Gastropub,3
197,NaN,4ad91beff964a5202e1821e3,Shaw's Crab House,41.889959,-87.627065,Seafood Restaurant,4
198,NaN,59deb72f061b514b32a062af,Gibson Italia,41.886606,-87.639593,Italian Restaurant,5


In [16]:
# Save the data to a csv so we do not need to run the code each time we start the program.
# This will save the number of API calls we need to make.
chicago_food_venues.to_csv('../saved_data/chicago_food_venues.csv')